In [1]:
import sys
sys.path.append('/home/miketheologitis/MapReduce-Implementation')

from mapreduce.cluster.local_cluster import LocalCluster

In [2]:
local_cluster = LocalCluster(
    n_workers=4,
    n_masters=2,
    initialize=True,
    verbose=False
)

In [3]:
local_monitoring = local_cluster.local_monitoring

In [4]:
import random
import string

def generate_random_string(str_len):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for _ in range(str_len))

def generate_random_string_list(n, str_len):
    return [generate_random_string(str_len) for _ in range(n)]

# Example usage
#data = generate_random_string_list(n=500000, str_len=20)
data = generate_random_string_list(n=100000, str_len=20)

In [5]:
data[:10]

['wrddhthhzwfiwqvqrkwd',
 'dmcfidzxusyornvxwazd',
 'wkkjirfbaxwwfggemqer',
 'bjuikrxxiioacjwfrtrk',
 'hduxqenlcxmjqxmayent',
 'suziwjcdorjbxafrryze',
 'wkxxzfvxnppfshkejudg',
 'zducsjaxonfqvcuryrmv',
 'mcewwwokcmhdixxvqjxc',
 'qjhdgrrujivbzgpmriao']

In [6]:
import sys
print(f"Size: {round(sys.getsizeof(data) / (1024 * 1024), 3)} MB")

Size: 0.764 MB


In [7]:
def map_func(data):
    result = []
    for string in data:
        for char in string:
            result.append((char, 1))
    return result

def reduce_func(data):
    return sum(data)

In [9]:
future1 = local_cluster.mapreduce(data, map_func, reduce_func)

In [9]:
future1

<Future at 0x7fc770d4ff50 state=running>

In [10]:
future2 = local_cluster.mapreduce(data, map_func, reduce_func)

In [11]:
future2

<Future at 0x7fc752d07910 state=running>

In [12]:
#!docker kill ed1374a13ab9 2e48d1b685cd 26849605aa1e 0d746726f73b

In [13]:
#!docker kill c3101d644816

In [10]:
local_monitoring.print_zoo()


----------------- Zoo Masters -----------------
Master aa95a62aa7a8 :  MasterInfo(state='nothing')
Master 67e1b07f99d7 :  MasterInfo(state='nothing')

----------------- Zoo Workers -----------------
Worker c85e3894b068 :  WorkerInfo(state='idle')
Worker 6bf8ca3658f7 :  WorkerInfo(state='idle')
Worker f9bce8bf5115 :  WorkerInfo(state='idle')
Worker 86bfa3b598f4 :  WorkerInfo(state='idle')

----------------- Zoo Map Tasks -----------------

----------------- Zoo Shuffle Tasks -----------------

----------------- Zoo Reduce Tasks -----------------

----------------- Zoo Jobs ---------------------
Job 0 :  Job(state='in-progress', requested_n_workers=None, master_hostname='aa95a62aa7a8')


------------- Dead Worker Tasks ---------------------


In [15]:
local_monitoring.print_hdfs('jobs')

job_0/
  data.pickle
  map_func.pickle
  map_results/
  map_tasks/
  reduce_func.pickle
  reduce_results/
  shuffle_results/
job_1/
  data.pickle
  map_func.pickle
  map_results/
  map_tasks/
  reduce_func.pickle
  reduce_results/
  shuffle_results/


In [40]:
local_cluster.scale(n_masters=2)

In [19]:
local_cluster.scale(n_workers=0)

In [21]:
local_cluster.clear()

In [31]:
local_cluster.shutdown_cluster(cleanup=True)

In [12]:
zk_client = local_monitoring.get_zk_client()

In [13]:
lock = zk_client.zk.Lock(path="/locks/master_job_assignment_lock", identifier='myself')

In [14]:
lock.contenders()

['67e1b07f99d7']